In [1]:
from keras.datasets import fashion_mnist
from art.utils import load_mnist
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Lambda, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential,load_model
from art.metrics.metrics import empirical_robustness, clever_t, clever_u, clever, loss_sensitivity, wasserstein_distance
import numpy as np
from art.estimators.classification.keras import KerasClassifier
import logging
from progressbar import ProgressBar

import sys
sys.path.append('..')  
from GradPri_utils.utils import *

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
tf.compat.v1.disable_eager_execution()

In [2]:
R_L1 = 40
R_L2 = 2
R_LI = 0.1

def _create_krclassifier_FMNIST():
    num_classes = 10
    input_shape = (28,28,1)
    
    model = Sequential()
    # Block 1
    model.add(Conv2D(64, (3, 3), padding='same', name = 'block1_conv1', 
                     input_shape=input_shape))
    # model.add(BatchNormalization(name = 'block1_bn1'))
    model.add(Activation('relu', name = 'block1_act1'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool1'))

    model.add(Conv2D(128, (3, 3), padding='same', name = 'block1_conv2'))
    # model.add(BatchNormalization(name = 'block1_bn2'))
    model.add(Activation('relu', name = 'block1_act2'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool2'))

    model.add(Conv2D(256, (3, 3), padding='same', name = 'block1_conv3'))
    # model.add(BatchNormalization(name = 'block1_bn3'))
    model.add(Activation('relu', name = 'block1_act3'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool3'))
    # top
    model.add(Flatten(name='flatten'))
    model.add(Dense(1024, activation='relu', name='fc1'))
    model.add(Dropout(0.5, name='drop1'))
    model.add(Dense(32, activation='relu', name='fc2'))
    model.add(Dropout(0.5, name='drop2'))
    # model.add(Dense(64, activation='relu', name='fc3'))
    # model.add(Dropout(0.5, name='drop3'))
    model.add(Dense(num_classes, activation='softmax', name='predictions'))

    
    # model = load_model("./fmnist_CNN.h5")
    opt = tf.keras.optimizers.Adam(learning_rate=0.00005)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])

    # Get the classifier
    krc = KerasClassifier(model=model, clip_values=(0, 1), use_logits=False)

    return krc

logger = logging.getLogger(__name__)

In [3]:
# Get Fashion-MNIST
(x_train, y_train), (x_test, y_test), _, _ = load_mnist()

# Get the classifier
# krc = _create_krclassifier_FMNIST()
# krc.fit(x_train, y_train, batch_size=256, nb_epochs=5, shuffle=True,
#         validation_data=(x_test[0:5000], y_test[0:5000]), verbose=1)

krc = load_model('./fmnist_CNN_art.h5')
krc = KerasClassifier(model=krc, clip_values=(0, 1), use_logits=False)

In [4]:
# krc.save(filename='fmnist_CNN_art.h5', path='.')

In [4]:
# Gini = []
# total_sample_num = 2000
# predicted_confidence = []

# pbar = ProgressBar()
# for index in pbar(range(total_sample_num)):
#     pre_tmp = krc.predict(np.reshape(x_test[index], [-1,28,28,1]))
#     Gini.append(1-np.sum(pre_tmp*pre_tmp))
#     predicted_confidence.append(pre_tmp)

/public/liujiawei/anaconda3/envs/ZHB_env/lib/python3.6/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
100% |########################################################################|


In [5]:
# indexs = np.argsort(Gini)
# indexs = indexs[::-1]
# # 计算APFD指标
# APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
#                    predicted_confidence=np.array(predicted_confidence))
# print("APFD: ", APFD)

# # 计算RAUC指标
# RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
#                     predicted_confidence=np.array(predicted_confidence))
# print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.9419964788732395
RAUC:  0.9585205891839587


In [17]:
Gini = []
total_sample_num = 2000
predicted_confidence = []

pbar = ProgressBar()
for index in pbar(range(total_sample_num)):
    pre_tmp = krc.predict(np.reshape(x_test[index], [-1,28,28,1]))
    Gini.append(
        np.sqrt(
            ( np.sum( (np.max(pre_tmp)-pre_tmp) * (np.max(pre_tmp)-pre_tmp) ) ) / len(pre_tmp[0])
        )
    )
    predicted_confidence.append(pre_tmp)

100% |########################################################################|


In [19]:
indexs = np.argsort(Gini)
# indexs = indexs[::-1]
# 计算APFD指标
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
                   predicted_confidence=np.array(predicted_confidence))
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=y_test[range(total_sample_num)],
                    predicted_confidence=np.array(predicted_confidence))
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.942637323943662
RAUC:  0.9591728487976203


In [140]:
# craft adversarial examples
x_adv = []
y_adv = []
max_iter = 100
eps = 50
thre_conf = 0.999999
pbar = ProgressBar()
for i in pbar(range(100)):
    pre_tmp = krc.predict(np.reshape(x_test[i], [-1,28,28,1]))
    x_adv_tmp = np.copy(x_test[i])
    # check whether x is right prediction
    if np.argmax(pre_tmp[0]) == np.argmax(y_test[i]):
        for j in range(max_iter):
            grad_tmp = krc.class_gradient(np.reshape(x_adv_tmp, [-1,28,28,1]), 
                                                 label=np.argmax(y_test[i]))[0][0]
            x_adv_tmp = x_adv_tmp - eps * grad_tmp
            pre_adv_tmp = krc.predict(np.reshape(x_adv_tmp, [-1,28,28,1]))
            if np.argmax(pre_adv_tmp[0]) != np.argmax(y_test[i]):
                if np.max(pre_adv_tmp[0])>thre_conf:
                    x_adv.append(x_adv_tmp)
                    y_adv.append(y_test[i])
                    break

100% |########################################################################|


In [20]:
# x_adv = np.array(x_adv)
# y_adv = np.array(y_adv)
# np.save('./x_adv.npy', x_adv)
# np.save('./y_adv.npy', y_adv)
x_adv = np.load('./x_adv.npy')
y_adv = np.load('./y_adv.npy')
print(x_adv.shape)

(15, 28, 28, 1)


In [21]:
X_adv_clean = np.concatenate((x_adv, x_test[100:150]))
Y_adv_clean = np.concatenate((y_adv, y_test[100:150]))

In [26]:
Gini_adv = []
predicted_confidence = []

pbar = ProgressBar()
for index in pbar(range(len(X_adv_clean))):
    pre_tmp = krc.predict(np.reshape(X_adv_clean[index], [-1,28,28,1]))
#     Gini_adv.append(
#         np.sqrt(
#             ( np.sum( (np.max(pre_tmp)-pre_tmp) * (np.max(pre_tmp)-pre_tmp) ) ) / len(pre_tmp[0])
#         )
#     )
    Gini_adv.append(1-np.sum(pre_tmp*pre_tmp))
    predicted_confidence.append(pre_tmp)

100% |########################################################################|


In [27]:
indexs = np.argsort(Gini_adv)
indexs = indexs[::-1]
# 计算APFD指标
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
                   predicted_confidence=np.array(predicted_confidence))
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
                    predicted_confidence=np.array(predicted_confidence))
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.13076923076923078
RAUC:  0.13793103448275862


In [28]:
# BoundaryGini = []
# predicted_confidence = []

# R_L1 = 40
# R_L2 = 2
# R_LI = 0.1
# pbar = ProgressBar()
# for index in pbar(range(len(X_adv_clean))):
#     pre_tmp = krc.predict(np.reshape(X_adv_clean[index], [-1,28,28,1]))
#     res_tmp = clever_u(krc, X_adv_clean[index], 10, 10, R_L2, norm=2, pool_factor=5, verbose=False)
#     BoundaryGini.append(np.min([res_tmp, 2]))
#     predicted_confidence.append(pre_tmp)

100% |########################################################################|


In [29]:
# indexs = np.argsort(BoundaryGini)
# # indexs = indexs[::-1]
# # 计算APFD指标
# APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
#                    predicted_confidence=np.array(predicted_confidence))
# print("APFD: ", APFD)

# # 计算RAUC指标
# RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
#                     predicted_confidence=np.array(predicted_confidence))
# print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.6364102564102564
RAUC:  0.7045977011494253


In [39]:
BoundaryGini = []
predicted_confidence = []

R_L1 = 40
R_L2 = 2
R_LI = 0.1
pbar = ProgressBar()
for index in pbar(range(len(X_adv_clean))):
    pre_tmp = krc.predict(np.reshape(X_adv_clean[index], [-1,28,28,1]))
#     res_tmp = clever_u(krc, X_adv_clean[index], 10, 10, R_L2, norm=2, pool_factor=5, verbose=False)
#     BoundaryGini.append(np.min([res_tmp, 2]))
    res_tmp = clever(krc, X_adv_clean[index], 10, 10, R_L2, norm=2, pool_factor=5, verbose=False)
    BoundaryGini.append(
        np.sqrt(
            ( np.sum( (np.min(res_tmp)-res_tmp) * (np.min(res_tmp)-res_tmp) ) ) / len(res_tmp)
        )
    )
    predicted_confidence.append(pre_tmp)

100% |########################################################################|


In [40]:
indexs = np.argsort(BoundaryGini)
# indexs = indexs[::-1]
# 计算APFD指标
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
                   predicted_confidence=np.array(predicted_confidence))
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
                    predicted_confidence=np.array(predicted_confidence))
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.5943589743589743
RAUC:  0.6574712643678161


In [204]:
grad_Gini = []
predicted_confidence = []
pbar = ProgressBar()
for index in pbar(range(len(X_adv_clean))):
    pre_tmp = krc.predict(np.reshape(X_adv_clean[index], [-1,28,28,1]))
    grad_tmp = krc.class_gradient(np.reshape(X_adv_clean[index], [-1,28,28,1]), 
                                  label=np.argmax(pre_tmp[0]))[0][0]
    grad_Gini.append(np.sum(np.abs(grad_tmp)))
    predicted_confidence.append(pre_tmp)

100% |########################################################################|


In [205]:
indexs = np.argsort(grad_Gini)
indexs = indexs[::-1]
# 计算APFD指标
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
                   predicted_confidence=np.array(predicted_confidence))
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=Y_adv_clean,
                    predicted_confidence=np.array(predicted_confidence))
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.13076923076923078
RAUC:  0.13793103448275862


In [13]:
clever_t?